# TODO:
https://www.kaggle.com/competitions/playground-series-s3e2/discussion/378795
https://www.kaggle.com/competitions/playground-series-s3e2/discussion/378780

These show that we should incorporate original data but when validating using K fold methods, we should only validate based on data in the competition dataset and not on original dataset. So implement this technique for this competition

### Also, bagging resulted in a lot better score with simple mean in the last competition, thought it didn't score much better on public, it absolutely ranked up to 60th position in the final private LB. So,
## Remember to trust your cvs over pbl

# Imports

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from pathlib import Path
import xgboost as xgb
import lightgbm as lgbm
import catboost
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score
from IPython.display import display
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
import optuna
from sklearn.preprocessing import StandardScaler

In [2]:
import warnings
warnings.filterwarnings('ignore')

# Loading Data

In [3]:
BASE_PATH = Path('../input/playground-series-s3e3')

# id is not going to be an informative feature, so we're dropping it for train
# but since we'll need test set's ids to make the submission file, so we'll save those in  a separate varible before dropping
train = pd.read_csv(BASE_PATH / "train.csv").drop(columns="id")
test = pd.read_csv(BASE_PATH / "test.csv")
test_idx = test.id
test = test.drop(columns="id")

# It's been shown that incorporating original data, improves scores - at least on the public leaderboard. So let's do that!
original = pd.read_csv('../input/ibm-hr-analytics-attrition-dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv')

train.head()

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EnvironmentSatisfaction,Gender,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,36,Travel_Frequently,599,Research & Development,24,3,Medical,1,4,Male,...,80,1,10,2,3,10,0,7,8,0
1,35,Travel_Rarely,921,Sales,8,3,Other,1,1,Male,...,80,1,4,3,3,4,2,0,3,0
2,32,Travel_Rarely,718,Sales,26,3,Marketing,1,3,Male,...,80,2,4,3,3,3,2,1,2,0
3,38,Travel_Rarely,1488,Research & Development,2,3,Medical,1,3,Female,...,80,0,15,1,1,6,0,0,2,0
4,50,Travel_Rarely,1017,Research & Development,5,4,Medical,1,2,Female,...,80,0,31,0,3,31,14,4,10,1


In [4]:
original.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


# Pre-Pre-Processing

### Let's make the feature names and order consistent b/w our competition dataset and original dataset, before we concatenate

In [5]:
original['Attrition'] = (original['Attrition'] == 'Yes').astype(np.int64)

# in original data, id is termed as "EmployeeNumber", so let's drop it
original.drop(columns="EmployeeNumber", inplace=True)

In [6]:
# now reordering the features in original dataset
original = original[list(train.columns)]

### Someone in the first compeition showed that adding a source feature i.e. a feature that indicates whether a given record is from original dataset or synthetic improves performance, so let's' do that!

In [7]:
original["is_original"] = 1
train["is_original"] = 0
test["is_original"] = 0

### Let's finally concatenate

In [8]:
train_extended = pd.concat([train, original]).reset_index(drop=True)
len(train_extended)

3147

### checking for null values

In [9]:
pd.concat([train_extended.isnull().sum().rename("Missing in Train"),
           test.isnull().sum().rename("Missing in Test")], axis=1).sort_values(by="Missing in Train")

,Missing in Train,Missing in Test
Age,0,0.0
Over18,0,0.0
OverTime,0,0.0
PercentSalaryHike,0,0.0
PerformanceRating,0,0.0
RelationshipSatisfaction,0,0.0
StandardHours,0,0.0
NumCompaniesWorked,0,0.0
StockOptionLevel,0,0.0
TrainingTimesLastYear,0,0.0


#### Insights: No missing values! Something to celebrate! :p

## Let's also concatenate test data to train

In [10]:
y = train_extended.Attrition
y

0       0
1       0
2       0
3       0
4       1
       ..
3142    0
3143    0
3144    0
3145    0
3146    0
Name: Attrition, Length: 3147, dtype: int64

In [11]:
df = pd.concat([train_extended.drop(columns="Attrition"), test])

# Preprocessing

### Identifying Categorical Features

In [12]:
df.dtypes.sort_values()

Age                          int64
YearsSinceLastPromotion      int64
YearsInCurrentRole           int64
YearsAtCompany               int64
WorkLifeBalance              int64
TrainingTimesLastYear        int64
TotalWorkingYears            int64
StockOptionLevel             int64
StandardHours                int64
RelationshipSatisfaction     int64
PerformanceRating            int64
PercentSalaryHike            int64
NumCompaniesWorked           int64
MonthlyRate                  int64
YearsWithCurrManager         int64
MonthlyIncome                int64
JobSatisfaction              int64
DailyRate                    int64
DistanceFromHome             int64
Education                    int64
EmployeeCount                int64
HourlyRate                   int64
EnvironmentSatisfaction      int64
JobLevel                     int64
JobInvolvement               int64
is_original                  int64
Gender                      object
MaritalStatus               object
OverTime            

### Remember, being of type int, doesn't mean that the feature cannot be categorial.
#### Let's check for unique values in each column

In [13]:
df.nunique().sort_values()

StandardHours                  1
EmployeeCount                  1
Over18                         1
is_original                    2
PerformanceRating              2
OverTime                       2
Gender                         2
BusinessTravel                 3
Department                     3
MaritalStatus                  3
RelationshipSatisfaction       4
JobSatisfaction                4
WorkLifeBalance                4
StockOptionLevel               5
JobInvolvement                 5
EnvironmentSatisfaction        5
Education                      6
JobLevel                       6
EducationField                 6
TrainingTimesLastYear          7
JobRole                        9
NumCompaniesWorked            11
PercentSalaryHike             15
YearsSinceLastPromotion       16
YearsWithCurrManager          18
YearsInCurrentRole            19
DistanceFromHome              29
YearsAtCompany                38
TotalWorkingYears             41
Age                           43
HourlyRate

#### INSIGHTS: Taking a quick look at number of unique values in features reveals that we should be safe setting the threshold for to 20 unique values for what consitutes as a categorical feature
#### We'll drop columns with only one value as they bring nothing to the table

#### But feel free to use your own intuition and test & trial to figure our what's works best in terms of threshold and features

In [14]:
feats_to_drop = [col for col in df.columns if df[col].nunique()==1]
cat_features = [col for col in df.columns if df[col].nunique() <= 20 and df[col].nunique() > 1]

In [15]:
df.drop(columns=feats_to_drop, inplace=True)

#### We won't use one hot encoder here, because we already have a large ratio of features to rows and one hotting would increase that ratio by a large margin even further which will result in severe overfitting
#### Rather we'll use ordinal/label encoder (they're basically the same thing)

In [16]:
ord_enc = OrdinalEncoder()

ord_enc.fit(df[cat_features])

df[cat_features] = ord_enc.transform(df[cat_features])
df.head()

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,HourlyRate,...,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,is_original
0,36,1.0,599,1.0,24,2.0,3.0,4.0,1.0,42,...,1.0,1.0,10,2.0,2.0,10,0.0,7.0,8.0,0.0
1,35,2.0,921,2.0,8,2.0,4.0,1.0,1.0,46,...,3.0,1.0,4,3.0,2.0,4,2.0,0.0,3.0,0.0
2,32,2.0,718,2.0,26,2.0,2.0,3.0,1.0,80,...,3.0,2.0,4,3.0,2.0,3,2.0,1.0,2.0,0.0
3,38,2.0,1488,1.0,2,2.0,3.0,3.0,0.0,40,...,2.0,0.0,15,1.0,0.0,6,0.0,0.0,2.0,0.0
4,50,2.0,1017,1.0,5,3.0,3.0,2.0,0.0,37,...,2.0,0.0,31,0.0,2.0,31,14.0,4.0,10.0,0.0


## Always a good idea to scale the features

In [17]:
# we don't want to scale the source column
sc = StandardScaler()
df_scaled = df.copy()
df_scaled.iloc[:, :-1] = pd.DataFrame(sc.fit_transform(df.iloc[:, :-1]), columns=df.columns[:-1])
df_scaled.head()

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,HourlyRate,...,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,is_original
0,-0.042187,-1.006941,-0.676304,-0.504308,1.914406,0.070936,0.631507,1.151976,0.786359,-1.276573,...,-0.680038,0.299291,-0.120890,-0.638312,0.351622,0.512223,-1.167739,1.553534,1.052484,0.0
1,-0.155733,0.556236,0.160523,1.478528,-0.108063,0.070936,1.402986,-1.590395,0.786359,-1.074371,...,1.182590,0.299291,-0.927119,0.188354,0.351622,-0.489294,-0.609548,-0.655171,-0.331151,0.0
2,-0.496370,0.556236,-0.367042,1.478528,2.167215,0.070936,-0.139973,0.237852,0.786359,0.644348,...,1.182590,1.526962,-0.927119,0.188354,0.351622,-0.656214,-0.609548,-0.339641,-0.607878,0.0
3,0.184905,0.556236,1.634065,-0.504308,-0.866489,0.070936,0.631507,0.237852,-1.271683,-1.377674,...,0.251276,-0.928379,0.550967,-1.464978,-2.630526,-0.155455,-1.167739,-0.655171,-0.607878,0.0
4,1.547454,0.556236,0.410011,-0.504308,-0.487276,1.059865,0.631507,-0.676271,-1.271683,-1.529326,...,0.251276,-0.928379,2.700910,-2.291644,0.351622,4.017536,2.739597,0.606946,1.605938,0.0


### Let's seprate test and train sets

In [18]:
X_train = df_scaled.iloc[:-len(test), :]
X_test = df_scaled.iloc[-len(test): , :]

# Modelling

### But first, let's setup cross validation

In [34]:
# for i, (x, y) in enumerate(zip([1,2,3], [4,5,6])):
#     print(f"{'*'*10} {i}")
#     print(f"X: {x}")
#     print(f"Y: {y}")    

********** 0
X: 1
Y: 4
********** 1
X: 2
Y: 5
********** 2
X: 3
Y: 6


In [36]:
# a = np.array([1,2,3])
# b = np.array([4,5,6])

# np.append(a, b)

array([1, 2, 3, 4, 5, 6])

In [19]:
# we're gonna train on the combined dataset but, we'll only calculate the validation score only on comp data

N_FOLDS = 10

def cross_validate(X, y, model):
    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=1337)
    all_scores = []
    
    X_comp = X[X.is_original==0]
    y_comp = y[X_comp.index]
    X_original = X[X.is_original==1]
    y_original = y[X_original.index]
    
    for fold_id, ((train_comp_idx, val_comp_idx), (train_org_idx, val_org_idx)) \
                in enumerate(zip(skf.split(X_comp, y_comp), skf.split(X_original, y_original))):
        
        # for training we'll use data from both datasets
        train_idx = np.append(train_comp_idx, train_org_idx)
        # but for validation we'll use only competition data
        val_idx = val_comp_idx
        
        
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y[train_idx], y[val_idx]
               
        model.fit(X_tr, y_tr)
        
        y_pred = model.predict_proba(X_val)[:, 1]
        
        auc = roc_auc_score(y_val, y_pred)
        
        print(f"Fold {fold_id} \t auc: {auc}")
        
        all_scores.append(auc)
    
    avg_auc = np.mean(all_scores)
    
    print(f"Avg AUC: {avg_auc}")

## INSIGHTS:
So one thing I've noticed is that using cross_validate this way gives us the score that's almost exactly the same that we get on public leaderboard. So let's use this method of cross validation to
* Tune all our models
* Select top k
* Take their predictions average
* submit

# Hyperparameters Tuning

## XGBoost

In [31]:
def objective_xgb(trial, X, y):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_loguniform('gamma', 0.00001, 0.5),
        'subsample': trial.suggest_loguniform('subsample', 0.2, 1.0),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.2, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 0.00001, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 0.00001, 1.0),
        'early_stopping_rounds': trial.suggest_int("early_stoppig_rounds", 10, 100)
    }
    

    # we're gonna train on the combined dataset but, we'll only calculate the validation score only on comp data

    N_FOLDS = 10

    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=1337)
    all_scores = []
    
    X_comp = X[X.is_original==0]
    y_comp = y[X_comp.index]
    X_original = X[X.is_original==1]
    y_original = y[X_original.index]
    
    for fold_id, ((train_comp_idx, val_comp_idx), (train_org_idx, val_org_idx)) \
                in enumerate(zip(skf.split(X_comp, y_comp), skf.split(X_original, y_original))):
        
        # for training we'll use data from both datasets
        train_idx = np.append(train_comp_idx, train_org_idx)
        # but for validation we'll use only competition data
        val_idx = val_comp_idx
        
        
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y[train_idx], y[val_idx]
        
        model = xgb.XGBClassifier(**params)
        model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=False)
        
        y_pred = model.predict_proba(X_val)[:, 1]
        
        auc = roc_auc_score(y_val, y_pred)
        
#         print(f"Fold {fold_id} \t auc: {auc}")
        
        all_scores.append(auc)
    
    avg_auc = np.mean(all_scores)
    
    print(f"Avg AUC: {avg_auc}")
    
    return avg_auc

In [32]:
study_xgb = optuna.create_study(study_name="xgboost_tuning", direction="maximize")
func = lambda trial: objective_xgb(trial, X_train, y)
study_xgb.optimize(func, n_trials=100)

[I 2023-01-17 15:39:02,893] A new study created in memory with name: xgboost_tuning
[I 2023-01-17 15:39:11,102] Trial 0 finished with value: 0.9459114726971869 and parameters: {'n_estimators': 128, 'max_depth': 4, 'learning_rate': 0.1377132655000192, 'min_child_weight': 4, 'gamma': 0.006910429856181178, 'subsample': 0.43557898641630843, 'colsample_bytree': 0.38295488364827396, 'reg_alpha': 0.06508849850885894, 'reg_lambda': 0.05075093144690056, 'early_stoppig_rounds': 45}. Best is trial 0 with value: 0.9459114726971869.


Avg AUC: 0.9459114726971869


[I 2023-01-17 15:39:23,071] Trial 1 finished with value: 0.9302619966905681 and parameters: {'n_estimators': 329, 'max_depth': 10, 'learning_rate': 0.17550539771537726, 'min_child_weight': 5, 'gamma': 0.45046264297483407, 'subsample': 0.2641198447719257, 'colsample_bytree': 0.38624445898097776, 'reg_alpha': 0.00487058094662315, 'reg_lambda': 0.0002809496155313422, 'early_stoppig_rounds': 50}. Best is trial 0 with value: 0.9459114726971869.


Avg AUC: 0.9302619966905681


[I 2023-01-17 15:39:31,754] Trial 2 finished with value: 0.9341756297113439 and parameters: {'n_estimators': 112, 'max_depth': 7, 'learning_rate': 0.022300404842103327, 'min_child_weight': 2, 'gamma': 0.07288652089204267, 'subsample': 0.2966874919771711, 'colsample_bytree': 0.6795157247885405, 'reg_alpha': 8.021502125464333e-05, 'reg_lambda': 0.0003981328004288705, 'early_stoppig_rounds': 66}. Best is trial 0 with value: 0.9459114726971869.


...

Avg AUC: 0.9637550560764847


[I 2023-01-17 16:12:13,233] Trial 98 finished with value: 0.9814772936201507 and parameters: {'n_estimators': 448, 'max_depth': 9, 'learning_rate': 0.22290807602502186, 'min_child_weight': 1, 'gamma': 0.0007243578280907595, 'subsample': 0.5616947500145373, 'colsample_bytree': 0.39715330581607244, 'reg_alpha': 2.6525220599178655e-05, 'reg_lambda': 0.40054912804554493, 'early_stoppig_rounds': 44}. Best is trial 95 with value: 0.9840308420665563.


Avg AUC: 0.9814772936201507


[I 2023-01-17 16:12:27,103] Trial 99 finished with value: 0.9719725592939877 and parameters: {'n_estimators': 450, 'max_depth': 9, 'learning_rate': 0.20768139160423302, 'min_child_weight': 2, 'gamma': 0.0009948889624499533, 'subsample': 0.5654274458108787, 'colsample_bytree': 0.5178585262143724, 'reg_alpha': 4.493459399335371e-05, 'reg_lambda': 0.27279634347114556, 'early_stoppig_rounds': 46}. Best is trial 95 with value: 0.9840308420665563.


Avg AUC: 0.9719725592939877


In [33]:
study_xgb.best_value

0.9840308420665563

In [34]:
study_xgb.best_params

{'n_estimators': 462,
 'max_depth': 9,
 'learning_rate': 0.24754839357747555,
 'min_child_weight': 1,
 'gamma': 0.0002768382878439927,
 'subsample': 0.4962635986679422,
 'colsample_bytree': 0.43112487341982364,
 'reg_alpha': 2.4326609397287665e-05,
 'reg_lambda': 0.21844284657376126,
 'early_stoppig_rounds': 45}

In [35]:
## INSIGHTS: Okay okay, the score looks like it's overfitting but there's only one way to find out. i.e. Submit.
# And honestly, we should test this before going on with tuning more models to see if this method is even working or not!

In [36]:
xgb_params = {'n_estimators': 462,
                 'max_depth': 9,
                 'learning_rate': 0.24754839357747555,
                 'min_child_weight': 1,
                 'gamma': 0.0002768382878439927,
                 'subsample': 0.4962635986679422,
                 'colsample_bytree': 0.43112487341982364,
                 'reg_alpha': 2.4326609397287665e-05,
                 'reg_lambda': 0.21844284657376126,
                 'early_stoppig_rounds': 45}

In [37]:
X_train_fr, X_val, y_train_fr, y_val = train_test_split(X_train, y, test_size=0.1, shuffle=True, random_state=1337,
                                                        stratify=y)

In [40]:
xgb_tuned_clf = xgb.XGBClassifier(**xgb_params)
xgb_tuned_clf.fit(X_train_fr, y_train_fr, eval_set=[(X_val, y_val)], verbose=False)

[16:19:03] WARNING: ../src/learner.cc:627: 
Parameters: { "early_stoppig_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.43112487341982364, early_stoppig_rounds=45,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0.0002768382878439927, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.24754839357747555,
              max_bin=256, max_cat_to_onehot=4, max_delta_step=0, max_depth=9,
              max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=462, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=2.4326609397287665e-05, ...)

In [41]:
xgb_tuned_preds = xgb_tuned_clf.predict_proba(X_test)[:, 1]

In [42]:
submission = pd.DataFrame({"id": test_idx, "Attrition": xgb_tuned_preds})
submission.head()

,id,Attrition
0,1677,0.021201
1,1678,0.001126
2,1679,0.000017
3,1680,0.000405
4,1681,0.947035


In [43]:
submission.to_csv("submission.csv", index=False)